# Homework Assignment #08. Neighbour Joining
Implement neighbour joining algorithm

In [1]:
import numpy as np

def calc_sum_dist_spilman(a, from_i, except_j):
    s = sum([a[from_i, k] for k in range(a.shape[1]) if a[from_i, k] != np.inf and k != except_j] 
            + [a[k, from_i] for k in range(a.shape[0]) if a[k, from_i] != np.inf and k != except_j])
    return s / (a.shape[0]-2)


def calc_sum_dist_wiki(a, i, j):
    n = a.shape[0]
    raw_s = sum([a[i, k] for k in range(n) if a[i, k] != np.inf] + [a[k, i] for k in range(n) if a[k, i] != np.inf])  
    col_s = sum([a[j, k] for k in range(n) if a[j, k] != np.inf] + [a[k, j] for k in range(n) if a[k, j] != np.inf])  
    d_ij = a[min(i, j), max(i, j)]
    score = d_ij * (a.shape[0]-2) - raw_s - col_s
    return score
    
    
def calc_dist_to_node(a, i, j):
    n = a.shape[0]
    row_sum = sum([a[i, k] for k in range(n) if a[i, k] != np.inf] + [a[k, i] for k in range(n) if a[k, i] != np.inf])
    col_sum = sum([a[j, k] for k in range(n) if a[j, k] != np.inf] + [a[k, j] for k in range(n) if a[k, j] != np.inf])  
    d_1 = 0.5 * a[i, j] + 1/(2*(n-2)) * (row_sum - col_sum)
    d_2 = a[i, j] - d_1
    return d_1, d_2
    
def nj(d_matrix, leaves_names, debug=False):
    """
    d_matrix (numpy.ndarray): distance matrix
    debug (bool): flag to pring intermediate info.
    
    return (str): tree in Newick format.
    """
    def display_fn(debug_flag):
        def foo(text):
            if debug_flag:
                print(text)
        return foo
    print_debug = display_fn(debug)
    
    n = len(leaves_names)
    newick = {k: {'val': None} for k in leaves_names}
    a = np.copy(d_matrix)
    a_new = np.copy(a)
    leaves = leaves_names[:]
    
    for i_n in range(len(leaves)-1):
        print_debug(f"\nSTEP {i_n+1}")
        
        # Create new matrix (unefficient but more clear)
        a = a_new
        # Use inf as super high scores as we will pick minimum value
        a_new = np.ones((n-1-i_n, n-i_n)) * np.inf
        
        # Find most close leaves
        i_min, j_min = 0, 0
        min_score = np.inf
        for i in range(a.shape[0]):
            for j in range(i+1, a.shape[1]):
                #score = a[i, j] - calc_sum_dist_spilman(a, i, j) - calc_sum_dist_spilman(a, j, i)
                score = calc_sum_dist_wiki(a, i, j)
                if score < min_score:
                    print_debug(f"New score at: {i}, {j} {score}")
                    min_score = score
                    i_min, j_min = i, j
        print_debug(f"Minimum score: {min_score} ({i_min}, {j_min})")
        
        # Let i_min always be smaller than j_min
        if i_min > j_min:
            i_min, j_min = j_min, i_min
        print_debug(f"Indices of min val: {i_min}, {j_min} ({leaves[i_min]} vs. {leaves[j_min]})")
        
        # Add leaves info to the newick format
        #val_i = 0.5 * (a[i_min, j_min] + calc_sum_dist_spilman(a, i_min, j_min) - calc_sum_dist_spilman(a, j_min, i_min))
        #val_j = 0.5 * (a[i_min, j_min] + calc_sum_dist_spilman(a, j_min, i_min) - calc_sum_dist_spilman(a, i_min, j_min))
        val_i, val_j = calc_dist_to_node(a, i_min, j_min)
        print_debug(f"Vals: {val_i,} {val_j}")
        if newick[leaves[i_min]]['val'] is None and newick[leaves[j_min]]['val'] is None:
            merged = f"({leaves[i_min]}:{val_i}, {leaves[j_min]}:{val_j})"
            new_key = leaves[i_min] + leaves[j_min]
            newick[new_key] = {'val': merged}
        elif newick[leaves[i_min]]['val'] is None:
            merged = f"({newick[leaves[j_min]]['val']}:{val_j}, {leaves[i_min]}:{val_i})"
            new_key = leaves[i_min] + leaves[j_min]
            newick[new_key] = {'val': merged}
        elif newick[leaves[j_min]]['val'] is None:
            merged = f"({newick[leaves[i_min]]['val']}:{val_i}, {leaves[j_min]}:{val_j})"
            new_key = leaves[i_min] + leaves[j_min]
            newick[new_key] = {'val': merged}
        else:
            merged = f"({newick[leaves[i_min]]['val']}:{val_i}, {newick[leaves[j_min]]['val']}:{val_j})"
            new_key = leaves[i_min] + leaves[j_min]
            newick[new_key] = {'val': merged}
        del newick[leaves[i_min]]
        del newick[leaves[j_min]]
    
        print_debug(f"Current newick: {newick}")
        
        # Change leaves' names
        leaves[i_min] = leaves[i_min] + leaves[j_min]
        leaves[j_min:-1] = leaves[j_min+1:]
        leaves = leaves[:-1]
        print_debug(f"Merged leaves: {leaves}")
        
        if len(leaves) == 2:
            if sorted([i_min, j_min]) == [0, 1]:
                ind_last = 2
            elif sorted([i_min, j_min]) == [0, 2]:
                ind_last = 1
            else:
                ind_last = 0
            score_last = a[i_min, ind_last] - val_i
            print_debug(f"Last tree: {ind_last}, {score_last}")
            
            ind_last -=1
            if newick[leaves[ind_last]]['val'] is None:
                merged = f"({newick[leaves[i_min]]['val']}:{val_i}, {leaves[ind_last]}:{score_last})"
                new_key = leaves[i_min] + leaves[ind_last]
                newick[new_key] = {'val': merged}
            else:
                merged = f"({newick[leaves[i_min]]['val']}:{val_i}, {newick[leaves[ind_last]]['val']}:{score_last})"
                new_key = leaves[i_min] + leaves[ind_last]
                newick[new_key] = {'val': merged}
            
                print_debug("End algorithm.")
            break

        # Create new reduced matrix
        a_new = np.ones((n-1-i_n, n-1-i_n)) * np.inf
        
        # Transfer old matrix values to the new one
        print_debug(f"Matrix:\n {a}")
        for i in range(n-1-i_n):
            if i in [i_min, j_min]:
                continue
            for j in range(i+1, n-i_n):
                if j in [i_min, j_min]:
                    continue
                # Shift elements backwards
                if j > i_min and i > j_min:
                    a_new[i-1, j-1] = a[i, j]
                elif j > j_min:
                    a_new[i, j-1] = a[i, j]
                else:
                    a_new[i, j] = a[i, j]
                            
        # let i be the index of the merged columns 
        
        # Calculate merged column
        for i in range(n-i_n):
            if i not in [i_min, j_min]:            
                score = 0.5 * (a[min(i_min, i), max(i_min, i)] + a[min(j_min, i), max(j_min, i)] - a[i_min, j_min])
                if i < j_min:
                    a_new[min(i, i_min), max(i, i_min)] = score
                else:
                    a_new[min(i-1, i_min), max(i-1, i_min)] = score
                
        print_debug(f"New matrix:\n {a_new}")
        
    return newick[new_key]['val']#newick[list(newick.keys())[0]]['val']

### Preliminary test with known output
<img width="200" alt="tree" src="https://user-images.githubusercontent.com/23639048/68604635-fc821380-04bb-11ea-89ac-36c012091b32.png">

In [2]:
leaves = ['a', 'b', 'c', 'd', 'e']
a = np.array([
    [np.inf, 5, 9, 9, 8],
    [np.inf, np.inf, 10, 10, 9],
    [np.inf, np.inf, np.inf, 8, 7],
    [np.inf, np.inf, np.inf, np.inf, 3],
    [np.inf, np.inf, np.inf, np.inf, np.inf]
])
print('NJ: ', nj(a, leaves, debug=False))

NJ:  ((((a:2.0, b:3.0):3.0, c:4.0):2.0, d:2.0):2.0, e:1.0)


### Test 1

In [3]:
leaves = ['a', 'b', 'c', 'd']
a = np.array([
    [np.inf, 16, 16, 10],
    [np.inf, np.inf, 8, 8],
    [np.inf, np.inf, np.inf, 4],
    [np.inf, np.inf, np.inf, np.inf]
])
print('NJ: ', nj(a, leaves, debug=False))

NJ:  (((a:10.0, d:0.0):2.0, b:5.0):2.0, c:3.0)


### Test 2

In [4]:
leaves = ['a', 'b', 'c', 'd', 'e', 'f']
a = np.array([
    [np.inf, 5, 4, 7, 6, 8],
    [np.inf, np.inf, 7, 10, 9, 11],
    [np.inf, np.inf, np.inf, 7, 6, 8],
    [np.inf, np.inf, np.inf, np.inf, 5, 9],
    [np.inf, np.inf, np.inf, np.inf, np.inf, 8],
    [np.inf, np.inf, np.inf, np.inf, np.inf, np.inf]
])
print('NJ: ', nj(a, leaves, debug=False))

NJ:  (((((a:1.0, b:4.0):1.0, c:2.0):1.0, f:5.0):1.0, d:3.0):1.0, e:2.0)
